In [1]:
from transformers import pipeline , AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
from utils import tokenize_hfmodel_inputs

/home/vamsi/Desktop/projects/AI projects/Testing_diff_models/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
low_compute=['mistralai/Mistral-7B-v0.3','google/gemma-2b','microsoft/phi-2','meta-llama/Llama-2-7b-hf','teknium/OpenHermes-2.5-Mistral-7B','meta-llama/Llama-3.1-8B','microsoft/Phi-3-mini-128k-instruct']
instruct_counter_part=['mistralai/Mistral-7B-Instruct-v0.3']

Text completions

In [ ]:
model_name="google/gemma-2b"

In [ ]:
sample_text="How to mak a sandwich"

model=AutoModelForCausalLM.from_pretrained(model_name)
tokens,tokeniser=tokenize_hfmodel_inputs(sample_text,model_name,'cpu')
x=tokens
for _ in range(50):
    ops=model(input_ids=x)
    logits=ops.logits
    probs=F.softmax(logits[:,-1,:],dim=-1)

    next_token=torch.multinomial(probs.view(probs.shape[1]),1)

    x=torch.cat([x,next_token.view(1,1)],dim=-1)
op=''
for i in x.tolist():
    op=op+tokeniser.decode(i)
print(op)

Chat

In [2]:
model_name="google/gemma-2b-it"

In [ ]:
sample_text="write a c program to print first 5 numbers of fibonacci series."
model=AutoModelForCausalLM.from_pretrained(model_name)
tokeniser=AutoTokenizer.from_pretrained(model)
messages = [{"role": "user", "content": sample_text}]
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt")
output_ids = model.generate(input_ids.input_ids,max_length=200,do_sample=True,temperature=0.7,top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)
print(response)

In [ ]:
messages.append({"role": "assistant", "content": response})
messages.append({"role": "user", "content": "But you did not write c=a+b anywhere"})

prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt")

output_ids = model.generate(input_ids.input_ids, max_length=1200, do_sample=True, top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)

print(response)
